In [71]:
import numpy as np
import pandas as pd


In [72]:
'''Pulling in data and adding colummns'''
read_file = pd.read_csv('311-service-requests-from-2010-to-present.csv', nrows = 50000, low_memory = False)
data = read_file.to_numpy()
print(data[0])

[45050101 '2019-12-01T02:04:01.000' nan 'DOT'
 'Department of Transportation' 'Street Condition' 'Pothole' nan 10001.0
 'WEST   29 STREET' 'WEST   29 STREET' '5 AVENUE' 'BROADWAY' nan nan
 'BLOCKFACE' 'NEW YORK' nan nan 'Open' nan
 'The Department of Transportation referred this complaint to the appropriate Maintenance Unit for repair.'
 '2019-12-01T02:04:01.000' '05 MANHATTAN' nan 'MANHATTAN' 987653.0
 210944.0 'UNKNOWN' 'Unspecified' 'MANHATTAN' nan nan nan nan nan nan nan
 40.745668482774114 -73.9877188309367
 '{\'longitude\': \'-73.9877188309367\', \'latitude\': \'40.745668482774114\', \'human_address\': \'{"address": "", "city": "", "state": "", "zip": ""}\'}'
 nan nan nan nan nan]


In [73]:
"'Deleting columns with little utility.'"
np.delete(data, 4, axis=1)
np.delete(data,7,axis=1)
columns = list(range(0,46))

In [74]:
data = pd.DataFrame(data, columns = columns)

In [75]:
data = data.drop(columns = [45,44, 43, 42, 41,40,37,36,35,28,24, 30,9,3,31,32,33,34,26,27,23,10,11,19,20,29,22,12,13,14,15,16,17,18,21], axis = 1)

In [76]:
data = data.rename(columns={0:"EventID", 25:"Borough", 1:"Date Created",2: "Date Resolved",4:"Agency", 5: "Complaint Type", 6: "Descriptor", 7:"Location Type", 8:"Zip", 38: "Longitude", 39: "Latitude"})

In [77]:
data = data[data['Date Resolved'].notna()]


In [78]:
# Convert string representations to datetime objects
data['Date Created'] = pd.to_datetime(data['Date Created'])
data['Date Resolved'] = pd.to_datetime(data['Date Resolved'])

# Calculate the time taken in hours for each row
data['Hours Taken'] = (data['Date Resolved'] - data['Date Created']).dt.total_seconds() / 3600
data['Time Elapsed'] = data['Hours Taken']


# Convert hours to days
data['Days Taken'] = data['Hours Taken'] / 24

'" We discovered an error in the data: Some Resolutions are filed as *before* the report created date."'
data = data[(data['Time Elapsed'].notna()) & (data['Time Elapsed'] > 0)]
# Print the result

data

,EventID,Date Created,Date Resolved,Agency,Complaint Type,Descriptor,Location Type,Zip,Borough,Longitude,Latitude,Hours Taken,Time Elapsed,Days Taken
34,45053975,2019-12-01 01:48:45,2019-12-01 01:55:41,New York City Police Department,Noise - Residential,Loud Music/Party,Residential Building/House,10003.0,MANHATTAN,40.7315,-73.985571,0.115556,0.115556,0.004815
35,45050211,2019-12-01 01:48:31,2019-12-01 01:54:32,New York City Police Department,Noise - Commercial,Loud Music/Party,Store/Commercial,10009.0,MANHATTAN,40.722316,-73.983214,0.100278,0.100278,0.004178
37,45050009,2019-12-01 01:48:20,2019-12-01 01:54:57,New York City Police Department,Noise - Commercial,Loud Music/Party,Store/Commercial,10003.0,MANHATTAN,40.731788,-73.989691,0.110278,0.110278,0.004595
66,45049638,2019-12-01 01:36:06,2019-12-01 01:54:10,New York City Police Department,Noise - Commercial,Loud Music/Party,Store/Commercial,10036.0,MANHATTAN,40.763233,-73.992863,0.301111,0.301111,0.012546
77,45051138,2019-12-01 01:31:56,2019-12-01 01:51:18,New York City Police Department,Noise - Commercial,Loud Music/Party,Club/Bar/Restaurant,10032.0,MANHATTAN,40.8354,-73.943889,0.322778,0.322778,0.013449
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49992,44997196,2019-11-22 16:59:09,2019-11-22 23:44:25,New York City Police Department,Illegal Parking,Blocked Hydrant,Street/Sidewalk,10021.0,MANHATTAN,40.76793,-73.952974,6.754444,6.754444,0.281435
49993,44996197,2019-11-22 16:58:54,2019-11-22 17:22:58,New York City Police Department,Noise - Vehicle,Engine Idling,Street/Sidewalk,11365.0,QUEENS,40.730099,-73.810908,0.401111,0.401111,0.016713
49994,44994987,2019-11-22 16:58:42,2019-11-26 08:42:05,Department of Housing Preservation and Develop...,HEAT/HOT WATER,ENTIRE BUILDING,RESIDENTIAL BUILDING,11211.0,BROOKLYN,40.717423,-73.958151,87.723056,87.723056,3.655127
49998,44994608,2019-11-22 16:58:26,2019-11-25 09:15:49,Department of Housing Preservation and Develop...,HEAT/HOT WATER,ENTIRE BUILDING,RESIDENTIAL BUILDING,10451.0,BRONX,40.815147,-73.920994,64.289722,64.289722,2.678738


In [79]:
from statistics import mean 
from statistics import mode

data = data.drop('Zip', axis=1) #drop zipcode
data['Location Type'] = data['Location Type'].fillna(mode(data['Location Type'].dropna()))
data['Descriptor'] = data['Descriptor'].fillna(mode(data['Descriptor'].dropna()))
data['Complaint Type'] = data['Complaint Type'].fillna(mode(data['Complaint Type'].dropna()))
data['Agency'] = data['Agency'].fillna(mode(data['Agency'].dropna()))
data['Longitude'] = data['Longitude'].fillna(mean(data['Longitude'].dropna()))
data['Latitude'] = data['Latitude'].fillna(mean(data['Latitude'].dropna()))

data_fit = data

In [80]:
# Here, we valuate the distribution among categories. 
agency = data['Agency'].value_counts().sort_index()
complaint_type = data['Complaint Type'].value_counts().sort_index()
descriptor = data['Descriptor'].value_counts().sort_index()
location_type = data['Location Type'].value_counts().sort_index()
borough = data['Borough'].value_counts().sort_index()


print(f"{agency}\n\n{complaint_type}\n\n{descriptor}\n\n{location_type}\n\n{borough}")

A - Bronx                                                    18
A - Brooklyn                                                 76
A - Canine Task Force Citywide                               25
A - Illegal Posting Staten Island, Queens and Brooklyn      106
A - Manhattan                                                27
A - Queens                                                   65
A - Staten Island                                            54
BCC - Bronx                                                  87
BCC - Brooklyn North                                        168
BCC - Brooklyn South                                        264
BCC - Manhattan                                              16
BCC - Queens East                                           136
BCC - Queens West                                            99
BCC - Staten Island                                         203
Bronx 03                                                     10
Bronx 04                                

In [81]:
#We're creating a copy of the original data to avoid copywarnings.
data_fit = data.copy()

#encode 'Borough'
boroughs = np.unique(data_fit['Borough'].dropna(), return_inverse=False)
for i in range(len(boroughs)):
    # replace each unique 'Borough' value with its corresponding index
    data_fit['Borough'][data_fit['Borough'] == boroughs[i]] = i

# print the DataFrame after encoding 'Borough'
print(data_fit)

# encode 'Complaint Type'
complaints = np.unique(data_fit['Complaint Type'].dropna(), return_inverse=False)
for i in range(len(complaints)):
    # replace each unique 'Complaint Type' value with its corresponding index
    data_fit['Complaint Type'][data_fit['Complaint Type'] == complaints[i]] = i

# encode 'Agency'
agencies = np.unique(data_fit['Agency'].dropna(), return_inverse=False)
for i in range(len(agencies)):
    # replace each unique 'Agency' value with its corresponding index
    data_fit['Agency'][data_fit['Agency'] == agencies[i]] = i

# encode 'Descriptor'
descriptors = np.unique(data_fit['Descriptor'].dropna(), return_inverse=False)
for i in range(len(descriptors)):
    # replace each unique 'Descriptor' value with its corresponding index
    data_fit['Descriptor'][data_fit['Descriptor'] == descriptors[i]] = i

# print the DataFrame after encoding 'Borough', 'Complaint Type', 'Agency', and 'Descriptor'
print(data_fit)

# encode 'Location Type'
locations = np.unique(data_fit['Location Type'].astype(str).dropna(), return_inverse=False)
for i in range(len(locations)):
    # replace each unique 'Location Type' value with its corresponding index
    data_fit['Location Type'][data_fit['Location Type'] == locations[i]] = i

data_fit

        EventID        Date Created       Date Resolved  \
34     45053975 2019-12-01 01:48:45 2019-12-01 01:55:41   
35     45050211 2019-12-01 01:48:31 2019-12-01 01:54:32   
37     45050009 2019-12-01 01:48:20 2019-12-01 01:54:57   
66     45049638 2019-12-01 01:36:06 2019-12-01 01:54:10   
77     45051138 2019-12-01 01:31:56 2019-12-01 01:51:18   
...         ...                 ...                 ...   
49992  44997196 2019-11-22 16:59:09 2019-11-22 23:44:25   
49993  44996197 2019-11-22 16:58:54 2019-11-22 17:22:58   
49994  44994987 2019-11-22 16:58:42 2019-11-26 08:42:05   
49998  44994608 2019-11-22 16:58:26 2019-11-25 09:15:49   
49999  44995640 2019-11-22 16:58:00 2019-11-22 21:00:00   

                                                  Agency  \
34                       New York City Police Department   
35                       New York City Police Department   
37                       New York City Police Department   
66                       New York City Police Depar

/var/folders/z0/0n0vj_d51hqb2b2tq_b6jr140000gn/T/ipykernel_974/1034550032.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_fit['Borough'][data_fit['Borough'] == boroughs[i]] = i
/var/folders/z0/0n0vj_d51hqb2b2tq_b6jr140000gn/T/ipykernel_974/1034550032.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_fit['Borough'][data_fit['Borough'] == boroughs[i]] = i
/var/folders/z0/0n0vj_d51hqb2b2tq_b6jr140000gn/T/ipykernel_974/1034550032.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable

        EventID        Date Created       Date Resolved Agency Complaint Type  \
34     45053975 2019-12-01 01:48:45 2019-12-01 01:55:41     39             60   
35     45050211 2019-12-01 01:48:31 2019-12-01 01:54:32     39             56   
37     45050009 2019-12-01 01:48:20 2019-12-01 01:54:57     39             56   
66     45049638 2019-12-01 01:36:06 2019-12-01 01:54:10     39             56   
77     45051138 2019-12-01 01:31:56 2019-12-01 01:51:18     39             56   
...         ...                 ...                 ...    ...            ...   
49992  44997196 2019-11-22 16:59:09 2019-11-22 23:44:25     39             48   
49993  44996197 2019-11-22 16:58:54 2019-11-22 17:22:58     39             62   
49994  44994987 2019-11-22 16:58:42 2019-11-26 08:42:05     27             41   
49998  44994608 2019-11-22 16:58:26 2019-11-25 09:15:49     27             41   
49999  44995640 2019-11-22 16:58:00 2019-11-22 21:00:00     29             91   

      Descriptor           

/var/folders/z0/0n0vj_d51hqb2b2tq_b6jr140000gn/T/ipykernel_974/1034550032.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_fit['Location Type'][data_fit['Location Type'] == locations[i]] = i
/var/folders/z0/0n0vj_d51hqb2b2tq_b6jr140000gn/T/ipykernel_974/1034550032.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_fit['Location Type'][data_fit['Location Type'] == locations[i]] = i
/var/folders/z0/0n0vj_d51hqb2b2tq_b6jr140000gn/T/ipykernel_974/1034550032.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.

,EventID,Date Created,Date Resolved,Agency,Complaint Type,Descriptor,Location Type,Borough,Longitude,Latitude,Hours Taken,Time Elapsed,Days Taken
34,45053975,2019-12-01 01:48:45,2019-12-01 01:55:41,39,60,214,28,2,40.731500,-73.985571,0.115556,0.115556,0.004815
35,45050211,2019-12-01 01:48:31,2019-12-01 01:54:32,39,56,214,32,2,40.722316,-73.983214,0.100278,0.100278,0.004178
37,45050009,2019-12-01 01:48:20,2019-12-01 01:54:57,39,56,214,32,2,40.731788,-73.989691,0.110278,0.110278,0.004595
66,45049638,2019-12-01 01:36:06,2019-12-01 01:54:10,39,56,214,32,2,40.763233,-73.992863,0.301111,0.301111,0.012546
77,45051138,2019-12-01 01:31:56,2019-12-01 01:51:18,39,56,214,9,2,40.835400,-73.943889,0.322778,0.322778,0.013449
...,...,...,...,...,...,...,...,...,...,...,...,...,...
49992,44997196,2019-11-22 16:59:09,2019-11-22 23:44:25,39,48,37,35,2,40.767930,-73.952974,6.754444,6.754444,0.281435
49993,44996197,2019-11-22 16:58:54,2019-11-22 17:22:58,39,62,144,35,3,40.730099,-73.810908,0.401111,0.401111,0.016713
49994,44994987,2019-11-22 16:58:42,2019-11-26 08:42:05,27,41,134,26,1,40.717423,-73.958151,87.723056,87.723056,3.655127
49998,44994608,2019-11-22 16:58:26,2019-11-25 09:15:49,27,41,134,26,0,40.815147,-73.920994,64.289722,64.289722,2.678738


In [82]:
from sklearn.model_selection import train_test_split
X = data_fit.drop('Time Elapsed',axis=1)
X = X.drop('Date Created',axis=1)
X = X.drop('EventID', axis =1)
X = X.drop('Date Resolved', axis=1)
X = X.drop('Hours Taken', axis=1)
X = X.drop('Days Taken',axis=1)

Y = data_fit[['Time Elapsed']].copy() 
X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size=0.2, random_state=42)

In [83]:
from sklearn.ensemble import GradientBoostingRegressor

model_gbr =  GradientBoostingRegressor().fit(X_train, y_train)
y_test_pred = model_gbr.predict(X_test) 
print(y_test_pred)

/Users/rh/anaconda3/lib/python3.11/site-packages/sklearn/ensemble/_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[ 3.99186188  4.29828035  4.68627376 ... 60.76444605  2.94474471
  3.79636538]


In [88]:
model_gbr.get_params()

{'alpha': 0.9,
 'ccp_alpha': 0.0,
 'criterion': 'friedman_mse',
 'init': None,
 'learning_rate': 0.1,
 'loss': 'squared_error',
 'max_depth': 3,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_iter_no_change': None,
 'random_state': None,
 'subsample': 1.0,
 'tol': 0.0001,
 'validation_fraction': 0.1,
 'verbose': 0,
 'warm_start': False}

In [90]:
from sklearn.metrics import r2_score

#Let's check out our different hyperparameters
df = pd.DataFrame([],columns=['N_Estimators', 'Learning Rate','Depth',"R2"])
for k in [50, 100,300]:
    for z in [0.01, 0.1, 0.3]:
        for q in [1, 2, 3,5,7]:
            model_gbr =  GradientBoostingRegressor(n_estimators=k, learning_rate=z, max_depth = q, random_state=0).fit(X_train, y_train)
            y_test_pred = model_gbr.predict(X_test) 
            r2 = r2_score(y_test, y_test_pred)
            df2 = pd.DataFrame([[k,z,q,r2]], columns = ['N_Estimators', 'Learning Rate','Depth',"R2"])
            df = pd.concat([df,df2])         
print(df.sort_values("R2", ascending = False))

/Users/rh/anaconda3/lib/python3.11/site-packages/sklearn/ensemble/_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/rh/anaconda3/lib/python3.11/site-packages/sklearn/ensemble/_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/rh/anaconda3/lib/python3.11/site-packages/sklearn/ensemble/_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/rh/anaconda3/lib/python3.11/site-packages/sklearn/ensemble/_gb.py:570: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samp

  N_Estimators  Learning Rate Depth        R2
0          300           0.30     7  0.732853
0          300           0.10     7  0.711626
0          300           0.30     5  0.705243
0          100           0.30     7  0.703394
0           50           0.30     7  0.679129
0          100           0.10     7  0.667795
0          300           0.10     5  0.658996
0          100           0.30     5  0.657224
0           50           0.10     7  0.642289
0          300           0.30     3  0.635714
0           50           0.30     5  0.626837
0          300           0.01     7  0.619298
0          100           0.10     5  0.613099
0          100           0.30     3  0.595482
0           50           0.10     5  0.594711
0          300           0.10     3  0.594522
0          300           0.30     2  0.589239
0           50           0.30     3  0.579989
0          300           0.01     5  0.574016
0          100           0.30     2  0.566750
0          100           0.10     